# How do I make a pandas DataFrame smaller and faster?

In [1]:
import pandas as pd

In [8]:
url = 'http://bit.ly/drinksbycountry'
drinks = pd.read_csv(url)

In [9]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [10]:
# The .info() method prints information about a DataFrame: index and column dtypes, non-null values and memory usage.
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


In [13]:
# By default, .info() method does an estimation of the memory usage. With the memory_usage= parameter,
# a real memory usage calculation is performed at the cost of computational resources.
drinks.info(memory_usage='deep') # memory_usage='deep' 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 30.4 KB


In [15]:
# The .memory_usage() method returns the memory usage of each column in bytes.
drinks.memory_usage()

Index                             80
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

In [16]:
# As before, if we want pandas to calculate the real memory consumption, including that of 'object' dtypes,
# we have to use a paramenter. 
drinks.memory_usage(deep=True) # deep=True

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

In [17]:
# Summing the memory usage of all the columns
drinks.memory_usage(deep=True).sum()

31176

### Whenever an object dtype column has a low count of unique values and it's using them over and over again we can  improve its performance by converting it to a category dtype.

In [22]:
# The continent column has only 6 unique values that are being used for 193 rows. 
print(sorted(drinks.continent.unique()))
drinks.shape

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']


(193, 6)

In [25]:
# Assigning object dtype to category dtype (inplace operation by default).
drinks['continent'] = drinks.continent.astype('category')
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

In [27]:
# Now the string objects are stored only once as a category and are being referenced by an integer as needed.
drinks.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

In [28]:
# We can see the continent column has shrinked its memory used from 12332 to 744 bytes.
drinks.memory_usage(deep=True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

In [35]:
# This strategy won't work as expected for the country column since it contains 192 unique values
drinks.country.describe()

count            193
unique           193
top       Mauritania
freq               1
Name: country, dtype: object

In [43]:
# Let's see how the memory increases from 12588 to 18094 bytes. That's because every string was stored only once since
# every country name is different, and now we are adding the storage of 192 different int that refer to 192 categories.
drinks['country'] = drinks.country.astype('category')
print(drinks.memory_usage(deep=True)) # added a space for readability '\n'
print('\n', drinks.country.cat.codes.head())
drinks.country.cat.categories

Index                              80
country                         18094
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

 0    0
1    1
2    2
3    3
4    4
dtype: int16


Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua & Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
       ...
       'United Arab Emirates', 'United Kingdom', 'Uruguay', 'Uzbekistan',
       'Vanuatu', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', length=193)